In [1]:
import numpy as np
#from matplotlib import pyplot as plt
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
#import cv2
import os
#from sklearn import utils
import json
from keras.optimizers import Adam
from keras.applications import VGG16
from keras.applications import InceptionV3
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Cargamos los tensores y otros datos necesarios para el modelo
X_train = np.load('/content/drive/MyDrive/Colab Notebooks/X_train.npy')
#X_test = np.load('/content/drive/MyDrive/Colab Notebooks/X_test.npy')
y_train = np.load('/content/drive/MyDrive/Colab Notebooks/y_train.npy')
#y_test = np.load('/content/drive/MyDrive/Colab Notebooks/y_test.npy')
target_names = ["buildings", "forest", "glacier", "mountain", "sea", "street"]
num_targets = len(target_names)

In [3]:
# CUARTO MODELO:
# Transfer learning (fine-tuning)

# Modelo basado en transfer learning: el modelo preentrenado sobre el que cargar los pesos de la red es libre, siempre y cuando se elija
# cualquiera deKeras (https://keras.io/api/applications/). Se puede escoger realizar tanto featureextraction como fine-tuning,
# aunque se valorará más positivamente (dadala complejidad de congelar capas de entrenamiento) realizar fine-tuning.

base_model = InceptionV3(weights='imagenet', # También se probó VGG16, pero InceptionV3 ofrecía mejores resultados
                   include_top=False, # Desactivamos la última capa densa de salida
                   input_shape=(150, 150, 3))

for layer in base_model.layers:
    print("Nombre layer {}".format(layer.name))
    print("\tEs Trainable ? {}".format(layer.trainable))


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Nombre layer input_layer
	Es Trainable ? True
Nombre layer conv2d
	Es Trainable ? True
Nombre layer batch_normalization
	Es Trainable ? True
Nombre layer activation
	Es Trainable ? True
Nombre layer conv2d_1
	Es Trainable ? True
Nombre layer batch_normalization_1
	Es Trainable ? True
Nombre layer activation_1
	Es Trainable ? True
Nombre layer conv2d_2
	Es Trainable ? True
Nombre layer batch_normalization_2
	Es Trainable ? True
Nombre layer activation_2
	Es Trainable ? True
Nombre layer max_pooling2d
	Es Trainable ? True
Nombre layer conv2d_3
	Es Trainable ? True
Nombre layer batch_normalization_3
	Es Trainable ? True
Nombre layer activation_3
	Es Trainable ? True
Nombre layer conv2d_4
	Es Trainable ? True
Nombre layer batch_normalization_4
	Es Trainable ? True
Nombre layer activation_4
	Es Trainable ? True
Nombre layer max_pooling2d_1
	Es Trainable ? True
Nombre layer conv2d_8
	Es Trainable ? True
Nombre layer batch_normalization_8
	Es

In [4]:
# Primero, se eligen las capas a entrenar:

layers_to_train = ["input_layer_2",
                   "conv2d", "batch_normalization", "activation"]
# Capas a entrenar:
  #- Capa de entrada
  #- Primer bloque convolucional

base_model.trainable = True

# Recorremos cada capa
for layer in base_model.layers:
    set_trainable = False

    # Si la capa está en la lista, se entrena:
    for l in layers_to_train:
        if layer.name == l:
            set_trainable = True
        else:
            pass

    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [5]:
# Comprobamos que los valores sean correctos:
for layer in base_model.layers:
    print("Nombre layer {} - Trainable {}".format(layer.name, layer.trainable))

Nombre layer input_layer - Trainable False
Nombre layer conv2d - Trainable True
Nombre layer batch_normalization - Trainable True
Nombre layer activation - Trainable True
Nombre layer conv2d_1 - Trainable False
Nombre layer batch_normalization_1 - Trainable False
Nombre layer activation_1 - Trainable False
Nombre layer conv2d_2 - Trainable False
Nombre layer batch_normalization_2 - Trainable False
Nombre layer activation_2 - Trainable False
Nombre layer max_pooling2d - Trainable False
Nombre layer conv2d_3 - Trainable False
Nombre layer batch_normalization_3 - Trainable False
Nombre layer activation_3 - Trainable False
Nombre layer conv2d_4 - Trainable False
Nombre layer batch_normalization_4 - Trainable False
Nombre layer activation_4 - Trainable False
Nombre layer max_pooling2d_1 - Trainable False
Nombre layer conv2d_8 - Trainable False
Nombre layer batch_normalization_8 - Trainable False
Nombre layer activation_8 - Trainable False
Nombre layer conv2d_6 - Trainable False
Nombre layer

In [6]:
model4 = Sequential([ #Añadimos las capas densas
    base_model,
    Flatten(),
    Dense(256, activation='relu'), # Capa densa 1
    Dense(num_targets, activation='softmax') # Capa de salida final
])

In [7]:
model4.compile(optimizer="adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
%%time

early_stopping = EarlyStopping(monitor='val_accuracy', patience=3)

hist_model4 = model4.fit(X_train, y_train,
                  batch_size = 32,
                  epochs = 15,
                  validation_split = 0.20,
                  callbacks  = [early_stopping])

Epoch 1/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 577s 5s/step - accuracy: 0.6904 - loss: 3.4059 - val_accuracy: 0.8038 - val_loss: 1.1572
Epoch 2/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 582s 5s/step - accuracy: 0.9033 - loss: 0.4118 - val_accuracy: 0.8681 - val_loss: 0.6593
Epoch 3/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 558s 5s/step - accuracy: 0.9140 - loss: 0.2976 - val_accuracy: 0.8681 - val_loss: 0.6066
Epoch 4/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 604s 5s/step - accuracy: 0.9490 - loss: 0.1465 - val_accuracy: 0.8873 - val_loss: 0.5649
Epoch 5/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 617s 5s/step - accuracy: 0.9632 - loss: 0.1191 - val_accuracy: 0.8884 - val_loss: 0.6734
Epoch 6/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 523s 5s/step - accuracy: 0.9700 - loss: 0.0871 - val_accuracy: 0.8850 - val_loss: 0.6761
Epoch 7/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 601s 5s/step - accuracy: 0.9820 - loss: 0.0521 - val_accuracy: 0.8985 - val_loss: 0.5680
Epoch 8/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 567s 5s/step - accuracy: 0.9942 - loss: 0.0206 - val_accu

In [9]:
model4.save('/content/drive/MyDrive/Colab Notebooks/model4.h5')
with open('/content/drive/MyDrive/Colab Notebooks/hist_model4.json', 'w') as f:
    json.dump(hist_model4.history, f)